In [ ]:
pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 868.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 12.6 MB/s eta 0:00:00


In [ ]:
pip install langgraph requests numpy

In [ ]:
from google.colab import userdata
userdata.get('SERPAPI_KEY')

'3f689675b1f3fe325ef1d5a3de24aa3323501039cc9a67f11eae148645812f8a'

In [ ]:
import langgraph
import requests
import numpy
print("Dependencies installed!")

Dependencies installed!


In [ ]:
import json
import random
import requests
import numpy as np
from langgraph.graph import StateGraph, END
from typing import TypedDict, Dict, Any
from google.colab import userdata

# State object to hold shared data
class BrandMateState(TypedDict):
    user_input: str
    tone: str
    target: str
    goal: str
    style_guide: Dict[str, str]
    topic: str
    seo_package: Dict[str, Any]
    post: str
    metrics: Dict[str, Any]
    feedback: str
    q_table: Dict[str, Any]

# 1. User Interaction Agent
def user_interaction_agent(state: BrandMateState) -> BrandMateState:
    state["tone"] = "bold"
    state["target"] = "startup founders"
    state["goal"] = "follower growth"
    print(f"Parsed input: Tone={state['tone']}, Target={state['target']}, Goal={state['goal']}")
    return state

# 2. Brand Identity Agent
def brand_identity_agent(state: BrandMateState) -> BrandMateState:
    state["style_guide"] = {
        "tone": state["tone"],
        "sentence_length": "short",
        "vocabulary": "assertive"
    }
    print(f"Style guide: {state['style_guide']}")
    return state

# 3. Content Strategist Agent
def content_strategist_agent(state: BrandMateState) -> BrandMateState:
    topics = [
        "What I learned pitching to 5 VCs last month",
        "Startup hiring: The silent killer of momentum"
    ]
    state["topic"] = topics[0]  # Fixed topic for simplicity
    print(f"Selected topic: {state['topic']}")
    return state

# 4. SEO Agent (Real SerpAPI)
def seo_agent(state: BrandMateState) -> BrandMateState:
    api_key = userdata.get('SERPAPI_KEY')
    query = state["target"] + " LinkedIn post"
    url = f"https://serpapi.com/search.json?q={query}&api_key={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        keywords = [data.get("related_searches", [{}])[0].get("query", "founder hacks")][:2]
        hashtags = [f"#{k.replace(' ', '')}" for k in keywords]
        state["seo_package"] = {
            "keywords": keywords,
            "hashtags": hashtags,
            "structure": "short sentences, strong CTA"
        }
    except Exception as e:
        print(f"SEO API error: {e}")
        state["seo_package"] = {
            "keywords": ["founder hacks", "VC tips"],
            "hashtags": ["#FounderHacks", "#VCTips"],
            "structure": "short sentences, strong CTA"
        }
    print(f"SEO package: {state['seo_package']}")
    return state

# 5. Content Generator Agent
def content_generator_agent(state: BrandMateState) -> BrandMateState:
    topic = state["topic"]
    tone = state["style_guide"]["tone"]
    keywords = state["seo_package"]["keywords"]
    hashtags = state["seo_package"]["hashtags"]
    post = f"{topic}! {tone.capitalize()} advice for startup founders: Use {keywords[0]} to win. Act fast! {' '.join(hashtags)} Follow for more!"
    state["post"] = post
    print(f"Generated post: {post}")
    return state

# 6. Post Editor Agent
def post_editor_agent(state: BrandMateState) -> BrandMateState:
    post = state["post"].replace("  ", " ")
    state["post"] = post + " #GrowYourBrand"
    print(f"Edited post: {state['post']}")
    return state

# 7. Publishing Agent
def publishing_agent(state: BrandMateState) -> BrandMateState:
    state["metrics"] = {
        "likes": random.randint(50, 200),
        "shares": random.randint(10, 50),
        "comments": random.randint(5, 20),
        "impressions": random.randint(500, 1500)
    }
    print(f"Posted to LinkedIn. Metrics: {state['metrics']}")
    return state

# 8. RL Feedback Agent (Q-learning)
def rl_feedback_agent(state: BrandMateState) -> BrandMateState:
    if "q_table" not in state:
        state["q_table"] = {
            "bold_short": 0.0,
            "bold_long": 0.0
        }
    current_state = f"{state['tone']}_short"
    action = current_state
    metrics = state["metrics"]
    reward = (metrics["likes"] * 0.5 + metrics["comments"] * 1.0 + metrics["shares"] * 0.7) / 100
    alpha, gamma = 0.1, 0.9
    state["q_table"][action] = (1 - alpha) * state["q_table"][action] + alpha * (reward + gamma * max(state["q_table"].values()))
    best_action = max(state["q_table"], key=state["q_table"].get)
    state["feedback"] = f"Reward: {reward:.2f}. Suggest: Use {best_action.replace('_', ' ')} for next post."
    print(f"Feedback: {state['feedback']}")
    print(f"Q-table: {state['q_table']}")
    return state

# Set up LangGraph
workflow = StateGraph(BrandMateState)
workflow.add_node("user_interaction", user_interaction_agent)
workflow.add_node("brand_identity", brand_identity_agent)
workflow.add_node("content_strategist", content_strategist_agent)
workflow.add_node("seo", seo_agent)
workflow.add_node("content_generator", content_generator_agent)
workflow.add_node("post_editor", post_editor_agent)
workflow.add_node("publishing", publishing_agent)
workflow.add_node("rl_feedback", rl_feedback_agent)

workflow.add_edge("user_interaction", "brand_identity")
workflow.add_edge("brand_identity", "content_strategist")
workflow.add_edge("content_strategist", "seo")
workflow.add_edge("seo", "content_generator")
workflow.add_edge("content_generator", "post_editor")
workflow.add_edge("post_editor", "publishing")
workflow.add_edge("publishing", "rl_feedback")
workflow.add_edge("rl_feedback", END)

workflow.set_entry_point("user_interaction")

# Compile and run
app = workflow.compile()
initial_state = {"user_input": "Create a high-engagement LinkedIn post, emotional tone, for women startup founders, to grow followers"}
result = app.invoke(initial_state)

# Print final output
print("\nFinal Output:")
print(f"Generated Post: {result['post']}")
print(f"Metrics: {result['metrics']}")
print(f"Feedback: {result['feedback']}")
print(f"Q-table: {result['q_table']}")

Parsed input: Tone=bold, Target=startup founders, Goal=follower growth
Style guide: {'tone': 'bold', 'sentence_length': 'short', 'vocabulary': 'assertive'}
Selected topic: What I learned pitching to 5 VCs last month
SEO package: {'keywords': ['Startup founders linkedin post examples'], 'hashtags': ['#Startupfounderslinkedinpostexamples'], 'structure': 'short sentences, strong CTA'}
Generated post: What I learned pitching to 5 VCs last month! Bold advice for startup founders: Use Startup founders linkedin post examples to win. Act fast! #Startupfounderslinkedinpostexamples Follow for more!
Edited post: What I learned pitching to 5 VCs last month! Bold advice for startup founders: Use Startup founders linkedin post examples to win. Act fast! #Startupfounderslinkedinpostexamples Follow for more! #GrowYourBrand
Posted to LinkedIn. Metrics: {'likes': 69, 'shares': 44, 'comments': 19, 'impressions': 547}
Feedback: Reward: 0.84. Suggest: Use bold short for next post.
Q-table: {'bold_short': 0

In [ ]:
from google.colab import userdata
import requests
api_key = userdata.get('SERPAPI_KEY')
url = f"https://serpapi.com/search.json?q=startup+founders&api_key={api_key}"
response = requests.get(url)
print(response.json().get("related_searches", [{}])[0].get("query", "No results"))

Startup founders list


In [ ]:
!pip install streamlit langgraph requests numpy pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.0 MB/s eta 0:00:00
